# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaにおける人間による好みの予測に関するコンペティションに取り組んでいます。主な目的は、チャットボットの応答に対するユーザーの好みを予測し、どちらのモデルが勝者であるかを判断することです。

ノートブックでは、以下の手法やライブラリが使用されています：

1. **データの取り込みと前処理**：
   - `pandas`を使って訓練データを読み込み、`winner`列を作成するためのカスタム関数`get_winner`を定義しています。この関数は、各行の応答がどちらのモデルによって勝たれたのかを確認します。

2. **データの可視化**：
   - `seaborn`と`matplotlib`を用いて、`winner`列の分布や応答の文字数の相関を可視化しています。

3. **特徴量エンジニアリング**：
   - モデルAとモデルBのそれぞれの応答の長さを新たな特徴量として追加し、これをモデルの学習に使用します。

4. **モデルの学習**：
   - `scikit-learn`の`MultiOutputClassifier`と`RandomForestClassifier`を用いて、応答の長さを特徴量としてモデルを訓練し、予測を行っています。

5. **評価と予測**：
   - 検証データに対する予測結果を得て、`log_loss`を利用してモデルの性能を評価しています。

6. **テストデータに対する予測**：
   - テストセットを読み込み、事前に定義した特徴量エンジニアリングの手順を適用し、最終的な予測結果を生成して`submission.csv`形式で提出用ファイルを作成しています。

全体として、ノートブックは人間の好みを予測するために、ランダムフォレスト分類器を中心とした機械学習手法と特徴量加工を駆使しており、特に応答の長さに着目した解析を行っている点が特徴です。

---


# 用語概説 
以下は、Jupyter Notebookにおける専門用語の解説です。特に初心者や実務経験の少ない方にとって、マイナーまたは特有の知識が必要となる用語に焦点を当てています。

1. **特徴量エンジニアリング**: データから有用な情報を抽出し、モデルに適した形に変換するプロセスを指します。このノートブックでは、応答の長さを特徴量として追加することが特徴量エンジニアリングの一例です。

2. **相関行列**: 複数の変数間における相関関係を示す行列です。この行列を使用することで、それぞれの特徴量が他の特徴量とどの程度関連しているかを視覚的に理解できます。

3. **ワンホットエンコーディング**: カテゴリカルデータを機械学習モデルが扱える数値形式に変換する手法です。各カテゴリを二進数のベクトルで表現し、特定のカテゴリのみが1となり、他が0となるようにします。このノートブックでは、'winner'変数をワンホットエンコーディングしています。

4. **MultiOutputClassifier**: Scikit-learnライブラリの一部で、複数のターゲット変数を持つ分類問題に対応できるラッパークラスです。複数の分類器を同時に使用し、それぞれのターゲット変数に対して予測を行います。このノートブックでは、RandomForestClassifierを用いています。

5. **predict_proba**: モデルの予測メソッドの一つで、クラスごとの予測確率を提供します。例えば、特定の応答がモデルAまたはモデルBの勝者である確率を得るために使用されます。

6. **ログ損失 (Log Loss)**: モデルの予測の品質を評価するための損失関数です。実際のクラスラベルと予測確率の間の不一致を測定し、低いほど良いモデルを示します。このコンペティションでは、モデルの性能評価に使われています。

7. **データフレーム (DataFrame)**: Pandasライブラリのデータ構造の一つで、表形式のデータを扱うことができます。行と列を持ち、複数の異なるデータ型を格納することが可能です。このノートブック内でのデータ操作は、全てPandasのDataFrameを通じて行われています。

8. **テストセット**: モデルの精度を評価するために持ち出されるデータのことです。トレーニング (学習) セットとは異なり、モデルの訓練には使用されず、主に性能評価のために使用されます。

これらの用語は、機械学習や深層学習の初心者が理解するのに役立つ基本的な概念であり、特にこのノートブックのコンテキストに関連しています。

---


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.multioutput import MultiOutputClassifier
import numpy as np

# データセットを読み込む
train_df = pd.read_csv('/kaggle/input/my-files/train.csv')

# 列名を確認する
print(train_df.columns)

# 'winner' 列が正しく作成されているか確認する
def get_winner(row):
    # モデルAが勝者の場合
    if row['winner_model_a'] == 1:
        return 'a'
    # モデルBが勝者の場合
    elif row['winner_model_b'] == 1:
        return 'b'
    # 引き分けの場合
    else:
        return 'tie'

# 'winner' 列を新たに作成する
train_df['winner'] = train_df.apply(get_winner, axis=1)

# 'winner' 列が作成されたことを確認するため最初の数行を表示
print(train_df.head())

# DataFrameに'winner'列が存在するか確認する
print('winner' in train_df.columns)

# 'winner' 変数の分布を可視化する
sns.countplot(data=train_df, x='winner')
plt.title("勝者の分布")
plt.xlabel("勝者")
plt.ylabel("カウント")
plt.show()

# 特徴量エンジニアリング: 各応答の長さを特徴量として追加
train_df['response_a_length'] = train_df['response_a'].apply(len)
train_df['response_b_length'] = train_df['response_b'].apply(len)

# 相関行列を表示する
correlation_matrix = train_df[['response_a_length', 'response_b_length']].corr()
sns.heatmap(correlation_matrix, annot=True)
plt.title("相関行列")
plt.show()

# モデルの学習のためにデータを準備
X = train_df[['response_a_length', 'response_b_length']]
y = train_df['winner']

# 目的変数をワンホットエンコーディングする
y = pd.get_dummies(y)

# データを学習用と検証用に分割する
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# モデルを初期化して学習する
model = MultiOutputClassifier(RandomForestClassifier())
model.fit(X_train, y_train)

# 予測を行う
y_pred = model.predict_proba(X_val)

# 配列のリストを単一の配列に変換する
y_pred = np.array([pred[:, 1] for pred in y_pred]).T

# モデルを評価する
loss = log_loss(y_val, y_pred)
print(f'ログ損失: {loss}')

# 提出ファイルを準備する
# テストセットを読み込む
test_df = pd.read_csv('/kaggle/input/my-files/test.csv')

# テストセットに対して特徴量エンジニアリングの手順を追加
test_df['response_a_length'] = test_df['response_a'].apply(len)
test_df['response_b_length'] = test_df['response_b'].apply(len)

# テストセットで予測を行う
test_pred = model.predict_proba(test_df[['response_a_length', 'response_b_length']])

# 配列のリストを単一の配列に変換する
test_pred = np.array([pred[:, 1] for pred in test_pred]).T

# 提出用に予測結果をフォーマットする
submission = pd.DataFrame({
    'id': test_df['id'],
    'winner_model_a': test_pred[:, 0],
    'winner_model_b': test_pred[:, 1],
    'winner_tie': test_pred[:, 2]
})

# 提出ファイルを保存する
submission.to_csv('submission.csv', index=False)